In [1]:
'''
Step 1
'''

# Q1: Some of the facilities charge a fee to members, but some do not.
# Write a SQL query to produce a list of the names of the facilities that do

q1 = '''
    SELECT name, membercost
    FROM Facilities
    WHERE membercost > 0
'''


# Q2: How many facilities do not charge a fee to members?

q2 = '''
    SELECT COUNT(*)
    FROM Facilities
    WHERE membercost = 0;
'''


# Q3: Write an SQL query to show a list of facilities that charge a fee to members,
# where the fee is less than 20% of the facility's monthly maintenance cost.
# Return the facid, facility name, member cost, and monthly maintenance of the
# facilities in question.

q3 = '''
    SELECT 
        facid,
        name,
        membercost,
        monthlymaintenance
    FROM Facilities
    WHERE membercost > 0
      AND membercost < monthlymaintenance * 0.2;
'''

# Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
# Try writing the query without using the OR operator.

q4 = '''
    SELECT *
    FROM Facilities
    WHERE facid BETWEEN 1 AND 5;
'''

# Q5: Produce a list of facilities, with each labelled as
# 'cheap' or 'expensive', depending on if their monthly maintenance cost is
# more than $100. Return the name and monthly maintenance of the facilities
# in question.

q5 = '''
    SELECT
        name,
        monthlymaintenance,
        CASE WHEN monthlymaintenance > 100 THEN 'expensive'
             ELSE 'cheap' 
             END AS pricerange
    FROM Facilities;
'''

# Q6: You'd like to get the first and last name of the last member(s)
# who signed up. Try not to use the LIMIT clause for your solution. */

q6 = '''
    SELECT
      firstname,
      surname
    FROM Members
    WHERE joindate = (SELECT MAX(joindate) FROM Members);
'''

# Q7: Produce a list of all members who have used a tennis court.
# Include in your output the name of the court, and the name of the member
# formatted as a single column. Ensure no duplicate data, and order by
# the member name.

q7 = '''
    SELECT DISTINCT
      m.surname||' '||m.firstname AS member_name,
      f.name AS court_name
    FROM Bookings AS b
        LEFT JOIN (SELECT facid, name FROM Facilities) AS f
            USING(facid)
        LEFT JOIN (SELECT * FROM `Members`) AS m
            USING(memid)
    WHERE f.name IN ('Tennis Court 1', 'Tennis Court 2')
    ORDER BY member_name;
'''

# Q8: Produce a list of bookings on the day of 2012-09-14 which
# will cost the member (or guest) more than $30. Remember that guests have
# different costs to members (the listed costs are per half-hour 'slot'), and
# the guest user's ID is always 0. Include in your output the name of the
# facility, the name of the member formatted as a single column, and the cost.
# Order by descending cost, and do not use any subqueries.

q8 = '''
    SELECT
        f.name AS facility_name,
        m.surname||' '||m.firstname AS member_name,
        (CASE WHEN b.memid = 0 THEN f.guestcost
            ELSE f.membercost END) * b.slots AS cost
    FROM Bookings AS b
          LEFT JOIN Facilities AS f USING(facid)
          LEFT JOIN Members AS m USING(memid)
    WHERE b.starttime LIKE '2012-09-14%'
      AND (CASE WHEN b.memid = 0 THEN f.guestcost
            ELSE f.membercost END) * b.slots > 30
    ORDER BY cost DESC;
'''

# Q9: This time, produce the same result as in Q8, but using a subquery.

q9 = '''
    SELECT
        f.name AS facility_name,
        (SELECT CONCAT(m.firstname, ' ', m.surname)
         FROM Members AS m
         WHERE b.memid = m.memid) AS member_name,
        (CASE WHEN b.memid = 0 THEN f.guestcost
              ELSE f.membercost END * b.slots) AS cost
    FROM (
        SELECT facid, memid, slots
        FROM Bookings 
        WHERE starttime LIKE '2012-09-14%'
        ) AS b
        LEFT JOIN Facilities AS f USING(facid)
    WHERE b.facid IN (
        SELECT facid
        FROM Facilities
        WHERE (CASE WHEN b.memid = 0 THEN f.guestcost
                    ELSE f.membercost END) * b.slots > 30)
    ORDER BY cost DESC;
'''

In [2]:
'''Preparation for Step 2'''

from sqlalchemy import create_engine
import os

database = 'sqlite_db_pythonsqlite.db'
if os.path.exists(database):
    engine = create_engine('sqlite:///'+database)
    print('Table names: ', ', '.join(engine.table_names()))
    con = engine.connect()
else:
    print('database doesn\'t exist.')

Table names:  Bookings, Facilities, Members


In [3]:
# Q10: Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue. Remember
# that there's a different cost for guests and members!

q = '''
    WITH monthly AS (
        SELECT
            f.name,
            strftime('%Y', b.starttime) AS year,
            strftime('%m', b.starttime) AS month,
            (SUM((CASE WHEN b.memid = 0 THEN f.guestcost 
                  ELSE f.membercost END) * b.slots) - f.monthlymaintenance
            ) AS monthly_revenue
        FROM Bookings AS b, Facilities AS f
        WHERE b.facid = f.facid
        GROUP BY 1, 2, 3
    )
    SELECT
        name AS facility_name,
        SUM(monthly_revenue) AS total_revenue
    FROM monthly
    GROUP BY 1
    HAVING total_revenue < 1000
    ORDER BY total_revenue DESC;
    '''

res = con.execute(q)
print(f'{res.keys()[0]:<17}: {res.keys()[1]}')
print('-' * 32)
for fac, rev in res.fetchall():
    print(f'{fac:<17}: ${rev:> 8.2f}'.replace('$-', '-$').replace('$ ', ' $'))

facility_name    : total_revenue
--------------------------------
Pool Table       :  $ 225.00
Snooker Table    :  $ 195.00
Table Tennis     :  $ 150.00


In [4]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */
q11 = '''
    SELECT DISTINCT
        m1.surname AS member_sur,
        m1.firstname AS member_first,
        m2.surname AS recommended_by_sur,
        m2.firstname AS recommended_by_first
    FROM Members AS m1, Members As m2
    WHERE m1.recommendedby = m2.memid
    ORDER BY 1, 2;
    '''

res11 = con.execute(q11)
keys11 = res11.keys()

print(f'{keys11[0]:20} : {keys11[1]:20} : {keys11[2]:20} : {keys11[3]:20}')
print('-'*89)
for row in res11.fetchall():
    print(f'{row[0]:20} : {row[1]:20} : {row[2]:20} : {row[3]:20}')

member_sur           : member_first         : recommended_by_sur   : recommended_by_first
-----------------------------------------------------------------------------------------
Bader                : Florence             : Stibbons             : Ponder              
Baker                : Anne                 : Stibbons             : Ponder              
Baker                : Timothy              : Farrell              : Jemima              
Boothe               : Tim                  : Rownam               : Tim                 
Butters              : Gerald               : Smith                : Darren              
Coplin               : Joan                 : Baker                : Timothy             
Crumpet              : Erica                : Smith                : Tracy               
Dare                 : Nancy                : Joplette             : Janice              
Genting              : Matthew              : Butters              : Gerald              
Hunt      

In [5]:
# Q12: Find the facilities with their usage by member, but not guests */
q12 = '''
    SELECT
        f.name AS facility_name,
        m.surname||' '||m.firstname AS member_name,
        COUNT(*) AS count
    FROM Bookings AS b
        INNER JOIN Facilities AS f USING(facid)
        INNER JOIN Members AS m USING(memid)
    WHERE b.memid != 0
    GROUP BY 1, 2
    ORDER BU 1, 2;
'''
res12 = con.execute(q12)
keys12 = res12.keys()
res12 = res12.fetchmany(20)  # Note I fetched only 20 for readability of this notebook.

print('Q12c: list the facilities usage by member')
for k in keys12: 
    print(f'{k:23} | ', end='')

print()
print('-' * (26*len(keys12)-1))

for row in res12:
    for c in row:
        print(f'{c:23} | ', end='')
    print()

Q12c: list the facilities usage by member
facility_name           | member_name             | count                   | 
-----------------------------------------------------------------------------
Badminton Court         | Bader Florence          |                       9 | 
Badminton Court         | Baker Anne              |                      10 | 
Badminton Court         | Baker Timothy           |                       7 | 
Badminton Court         | Boothe Tim              |                      12 | 
Badminton Court         | Butters Gerald          |                      20 | 
Badminton Court         | Crumpet Erica           |                       2 | 
Badminton Court         | Dare Nancy              |                      10 | 
Badminton Court         | Farrell Jemima          |                       7 | 
Badminton Court         | Hunt John               |                       2 | 
Badminton Court         | Jones David             |                       8 | 
Badminton C

In [6]:
# Q13: Find the facilities usage by month, but not guests */

q13 = '''
    SELECT
        f.name AS facility_name,
        strftime('%m', b.starttime) AS month,
        COUNT(*) AS count
    FROM Bookings AS b
        LEFT JOIN Facilities AS f USING(facid)
    WHERE b.memid != 0
    GROUP BY 1, 2;
    '''

res13 = con.execute(q13)
keys13 = res13.keys()
res13 = res13.fetchall()

if res13:
    for k in keys13:
        print(f'{k:15} | ', end='')
    print()
    print('-'*(18*len(keys13)-1))

    for row in res13:
        for c in row:
            print(f'{c:>15} | ', end='')
        print()


facility_name   | month           | count           | 
-----------------------------------------------------
Badminton Court |              07 |              51 | 
Badminton Court |              08 |             132 | 
Badminton Court |              09 |             161 | 
 Massage Room 1 |              07 |              77 | 
 Massage Room 1 |              08 |             153 | 
 Massage Room 1 |              09 |             191 | 
 Massage Room 2 |              07 |               4 | 
 Massage Room 2 |              08 |               9 | 
 Massage Room 2 |              09 |              14 | 
     Pool Table |              07 |             103 | 
     Pool Table |              08 |             272 | 
     Pool Table |              09 |             408 | 
  Snooker Table |              07 |              68 | 
  Snooker Table |              08 |             154 | 
  Snooker Table |              09 |             199 | 
   Squash Court |              07 |              23 | 
   Squash C